<a href="https://colab.research.google.com/github/microsoft/qlib/blob/main/examples/workflow_by_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  Copyright (c) Microsoft Corporation.
#  Licensed under the MIT License.

In [1]:
import sys, site
from pathlib import Path

################################# NOTE #################################
#  Please be aware that if colab installs the latest numpy and pyqlib  #
#  in this cell, users should RESTART the runtime in order to run the  #
#  following cells successfully.                                       #
########################################################################

try:
    import qlib
except ImportError:
    # install qlib
    ! pip install --upgrade numpy
    ! pip install pyqlib
    # reload
    site.main()

scripts_dir = Path.cwd().parent.joinpath("scripts")
if not scripts_dir.joinpath("get_data.py").exists():
    # download get_data.py script
    scripts_dir = Path("~/tmp/qlib_code/scripts").expanduser().resolve()
    scripts_dir.mkdir(parents=True, exist_ok=True)
    import requests
    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/get_data.py") as resp:
        with open(scripts_dir.joinpath("get_data.py"), "wb") as fp:
            fp.write(resp.content)

In [6]:

import qlib
import pandas as pd
from qlib.config import REG_CN, REG_US
# from qlib.contrib.model.gbdt import LGBModel
# from qlib.contrib.data.handler import Alpha158
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict


In [11]:
# use default data
# NOTE: need to download data from remote: python scripts/get_data.py qlib_data_cn --target_dir ~/.qlib/qlib_data/cn_data
provider_uri = "/home/us_data"  # target_dir
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    sys.path.append(str(scripts_dir))
    from get_data import GetData
    GetData().qlib_data(target_dir=provider_uri, region=REG_US)
qlib.init(provider_uri=provider_uri, region=REG_US)

Qlib data is not found in /home/us_data


2021-10-16 06:14:50.026 | WARNING  | qlib.tests.data:_download_data:55 - The data for the example is collected from Yahoo Finance. Please be aware that the quality of the data might not be perfect. (You can refer to the original data source: https://finance.yahoo.com/lookup.)
2021-10-16 06:14:50.027 | INFO     | qlib.tests.data:_download_data:57 - qlib_data_us_1d_latest.zip downloading......
450095104it [00:18, 24886551.94it/s]                               
2021-10-16 06:15:08.121 | WARNING  | qlib.tests.data:_unzip:80 - will delete the old qlib data directory(features, instruments, calendars, features_cache, dataset_cache): /home/us_data


Will be deleted: 
	['/home/us_data/features', '/home/us_data/calendars', '/home/us_data/instruments']
If you do not need to delete /home/us_data, please change the <--target_dir>
Are you sure you want to delete, yes(Y/y), no (N/n): y


2021-10-16 06:15:14.620 | WARNING  | qlib.tests.data:_delete_qlib_data:105 - delete: /home/us_data/features
2021-10-16 06:15:16.468 | WARNING  | qlib.tests.data:_delete_qlib_data:105 - delete: /home/us_data/calendars
2021-10-16 06:15:16.470 | WARNING  | qlib.tests.data:_delete_qlib_data:105 - delete: /home/us_data/instruments
2021-10-16 06:15:16.472 | INFO     | qlib.tests.data:_unzip:83 - /home/us_data/20211016061449_qlib_data_us_1d_latest.zip unzipping......
100%|██████████████████████| 71959/71959 [00:16<00:00, 4255.66it/s]
[346:MainThread](2021-10-16 06:15:34,183) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[346:MainThread](2021-10-16 06:15:34,184) WARNING - qlib.Initialization - [config.py:412] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[346:MainThread](2021-10-16 06:15:34,187) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[346:Mai

In [12]:
market = "all"
# benchmark = "SH000300"

# train model

In [31]:
import ruamel.yaml as yaml
config_file="./benchmarks/TRA/configs/config_alstm.yaml"
with open(config_file) as f:
    config = yaml.safe_load(f)
# seed_suffix = ""
# config["task"]["model"]["kwargs"].update(
#     {"seed": seed, "logdir": config["task"]["model"]["kwargs"]["logdir"] + seed_suffix}
# )
config

{'qlib_init': {'provider_uri': '/home/us_data', 'region': 'REG_US'},
 'data_loader_config': {'class': 'StaticDataLoader',
  'module_path': 'qlib.data.dataset.loader',
  'kwargs': {'config': None}},
 'model_config': {'input_size': 16,
  'hidden_size': 256,
  'num_layers': 2,
  'num_heads': 2,
  'use_attn': True,
  'dropout': 0.1},
 'num_states': 1,
 'tra_config': {'num_states': 1,
  'hidden_size': 16,
  'tau': 1.0,
  'src_info': 'LR_TPE'},
 'task': {'model': {'class': 'TRAModel',
   'module_path': '/home/qlib_test/examples/benchmarks/TRA/src/model.py',
   'kwargs': {'lr': 0.0002,
    'n_epochs': 500,
    'max_steps_per_epoch': 100,
    'early_stop': 20,
    'seed': 1000,
    'logdir': 'output/test/alstm',
    'model_type': 'LSTM',
    'model_config': {'input_size': 16,
     'hidden_size': 256,
     'num_layers': 2,
     'num_heads': 2,
     'use_attn': True,
     'dropout': 0.1},
    'tra_config': {'num_states': 1,
     'hidden_size': 16,
     'tau': 1.0,
     'src_info': 'LR_TPE'},
   

In [ ]:
###################################
# train model
###################################
# data_handler_config = {
#     "start_time": "2008-01-01",
#     "end_time": "2020-08-01",
#     "fit_start_time": "2008-01-01",
#     "fit_end_time": "2014-12-31",
#     "instruments": market,
# }
# task = {
#     "model": {
#         "class": "TRAModel",
#         "module_path": "qlib.contrib.model.pytorch_tra",
#         "kwargs": {
#             # "loss": "mse",
#             # "colsample_bytree": 0.8879,
#             # "learning_rate": 0.0421,
#             # "subsample": 0.8789,
#             # "lambda_l1": 205.6999,
#             # "lambda_l2": 580.9768,
#             # "max_depth": 8,
#             # "num_leaves": 210,
#             # "num_threads": 20,
#         },
#     },
#     "dataset": {
#         "class": "DatasetH",
#         "module_path": "qlib.data.dataset",
#         "kwargs": {
#             "handler": {
#                 "class": "Alpha158",
#                 "module_path": "qlib.contrib.data.handler",
#                 "kwargs": data_handler_config,
#             },
#             "segments": {
#                 "train": ("2008-01-01", "2014-12-31"),
#                 "valid": ("2015-01-01", "2016-12-31"),
#                 "test": ("2017-01-01", "2021-10-31"),
#             },
#         },
#     },
# }

# model initiaiton
model = init_instance_by_config(config["task"]["dataset"])
dataset = init_instance_by_config(config["task"]["model"])

In [ ]:
# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id

# prediction, backtest & analysis

In [ ]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.model_strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": "2017-01-01",
        "end_time": "2020-08-01",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name="backtest_analysis"):
    recorder = R.get_recorder(recorder_id=rid, experiment_name="train_model")
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # backtest & analysis
    par = PortAnaRecord(recorder, port_analysis_config, "day")
    par.generate()

# analyze graphs

In [7]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
pred_df_dates = pred_df.index.get_level_values(level='datetime')
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

{'class': 'Recorder', 'id': '3d7490f78e644c0d8189b3aa3de0e995', 'name': 'mlflow_recorder', 'experiment_id': '3', 'start_time': '2021-10-04 06:22:46', 'end_time': '2021-10-04 06:23:52', 'status': 'FINISHED'}


## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

## analysis model

In [10]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ['label']

### score IC

In [ ]:
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)